In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

Using TensorFlow backend.


In [10]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    ds = pydicom.dcmread(filename) 
    print('Load file {} ...'.format(filename))
    if (ds.BodyPartExamined=='CHEST') and ((int(ds.PatientAge)<100) or (int(ds.PatientAge)>0))and (ds.PatientPosition in ['PA','AP']):
        img = ds.pixel_array
        return img
    else:
         print('Wrong enties in {}. Cant be used for modeling'.format(filename))
        
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    
    proc_img=(img-img_mean)/img_std
    r,c=img.shape
    ctr=(r/2-1,c/2-1)
    proc_img=img[int(ctr[0]-img_size[1]/2):int(ctr[0]+img_size[1]/2),int(ctr[1]-img_size[2]/2):int(ctr[1]+img_size[2]/2)]
    proc_img = color.gray2rgb(proc_img)
    proc_img=proc_img.reshape(img_size)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    file=open(model_path,'r')
    model=model_from_json(file.read())
    
    file.close()
    model.load_weights(weight_path)
    #'Pneumonia_my_model.best.hdf5'
    #'my_model.json'
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo 
    pred_y=model.predict(img)
    if pred_y>thresh:
        prediction='Model predicts that the patient has Pneumonia'
    else:
        prediction='Model predicts that the patient doesnt have Pneumonia'
    return prediction 

In [11]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model_final.json'
weight_path = 'Pneumonia_my_model.best_final.hdf5'

IMG_SIZE=(1,224,224,3) 
img_mean =0 # During training,model without mean centered images performed better on validation set; hence images were not mean cenetere
img_std = 1# During training,images were not divided by standard deviation

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.20591968 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Model predicts that the patient doesnt have Pneumonia
Load file test2.dcm ...
Model predicts that the patient doesnt have Pneumonia
Load file test3.dcm ...
Model predicts that the patient doesnt have Pneumonia
Load file test4.dcm ...
Wrong enties in test4.dcm. Cant be used for modeling
Load file test5.dcm ...
Model predicts that the patient doesnt have Pneumonia
Load file test6.dcm ...
Wrong enties in test6.dcm. Cant be used for modeling
